In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/mnt/data4/haryoaw_workspace/projects/2021_2/mbekk/idrecibrew2


In [2]:
import pandas as pd

In [3]:
from datasets import Dataset

In [69]:
from idrecibrew2.data.indonlg_tokenizer.tokenizer import IndoNLGTokenizer
from transformers import DataCollatorForSeq2Seq

In [6]:
tokenizer = IndoNLGTokenizer.from_pretrained("indobenchmark/indobart-v2")

In [13]:
df = pd.read_csv("data/processed/test.csv")

In [75]:
data = Dataset.from_pandas(df.iloc[0:10])

In [76]:
tokenizer.tokenize("Testing yah")

['▁', '<0x54>', 'esting', '▁yah']

In [94]:
column_source = 'src'
column_target = 'tgt'

In [122]:
max_input_length = 128
max_target_length = 256

def preprocess_function(examples):
    inputs = examples['src']
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["tgt"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [123]:
def process_data(example_data, idx):
    tokenized_input = {}
    tokenized_input.update({key: dict(tokenizer(example_data[key]))['input_ids'] for key in [column_source, column_target]})
    return tokenized_input

In [124]:
hasil = data.map(preprocess_function, with_indices=False, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [126]:
hasil

Dataset({
    features: ['attention_mask', 'basic_ingredient', 'input_ids', 'labels', 'no', 'src', 'tgt'],
    num_rows: 10
})

In [127]:
hasil = hasil.remove_columns(['no', 'basic_ingredient', 'src', 'tgt'])

In [128]:
hasil

Dataset({
    features: ['attention_mask', 'input_ids', 'labels'],
    num_rows: 10
})

In [129]:
from torch.utils.data import DataLoader

In [131]:
collator = DataCollatorForSeq2Seq(tokenizer, padding=True)

In [134]:
coba = DataLoader(hasil, 4, collate_fn = collator)

In [135]:
next(iter(coba))

{'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'input_ids': tensor([[    0, 25270, 12338,  3353,     2,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [    0,   486,  5957,  2681,   306, 39974, 16576,   299, 33716, 12338,
          7727,   327,   244,   163,   139,   178,   244,   163,   139,   173,
         39965,     2],
        [    0,  3569,  3793, 12338, 19340,  4046, 24919,     2,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [    0,   771,   459, 12338,     2,     1,     1,     1,     1,     1,
             1,     1,     1,     1, 